#  Experimental Design: Structural Misspecification in Choice Models



## Experiment Overview

This notebook evaluates the robustness and performance of discrete choice models under different types of model misspecification. We explore two key scenarios:

---

## **Scenario 1: Structural Misspecification**

We begin with a simple baseline logistic regression model and assess the impact of adding one structural component at a time. This helps us understand how each feature affects model performance, calibration, and posterior uncertainty.

### Components added individually:

- **Group-level effect**  
  Models unobserved heterogeneity by allowing group-specific intercepts or coefficients.

- **Interaction term**  
  Captures multiplicative relationships between features (e.g., $x_0 \cdot x_1$).

- **Higher-order nonlinear term**  
  Introduces nonlinearity (e.g., $x_0^2$) to model more complex decision surfaces.

We compare models based on predictive accuracy, uncertainty quantification, and calibration to evaluate how structural misspecification influences inference.

---

##  **Scenario 2: Misspecified Error Distributions**

In this scenario, we fix the model structure and vary the error distribution in the data-generating process to simulate noise misspecification. The model is always fit using a logistic (Gumbel-distributed error) likelihood, but the true data-generating errors differ.

###  Error distributions considered:

- **Normal (probit-like behavior)**  
  Well-behaved and symmetric.

- **Student's t (df=3)**  
  Heavy-tailed distribution, capturing extreme values.

- **Cauchy**  
  Extremely heavy-tailed with undefined moments — models extreme outliers.

- **Normal-contaminated**  
  Mixture of 90% standard normal and 10% noise with large variance (e.g., $\mathcal{N}(0, 25)$).

By fitting the same model to data with these different noise structures, we assess robustness of posterior inference, calibration, and classification accuracy under error term misspecification.

---

Each scenario is designed to isolate specific sources of model misspecification and test the effectiveness of Bayesian and Generalized Bayesian approaches under controlled deviations from the ideal model assumptions.





##  Experimental Setup



###  True Data Generating Process (DGP)

The data is generated using a utility model that includes **all three structural components**:

$u_i = X_i \beta_{g[i]} + \gamma_1 (x_{i0} \cdot x_{i1}) + \gamma_2 x_{i0}^2 + z_i, \quad z_i \sim \mathcal{N}(0, 1)
$, $
y_i = \mathbb{1}(u_i > 0)
$

- $X_i$: observed features
- $\beta_{g[i]}$: group-specific coefficients
- $z_i$: latent noise

---



###  Model Variants

Each model begins from the same **baseline model (M0)** and adds only one component at a time:

| Model Name | Included Components |
|------------|---------------------|
| **M0** (Baseline) | Linear terms only, no group structure |
| **M1** | M0 + Group-level effect |
| **M2** | M0 + Interaction term $(x_0 \cdot x_1)$ |
| **M3** | M0 + Nonlinear term $(x_0^2)$ |
| **Mf** | Model with every term |

This structure allows us to isolate the effect of each component.

---


##  Methodology



###  Dataset Description

The dataset simulates a realistic marketing scenario where customers are exposed to targeted campaigns. Each row corresponds to an individual customer and includes both behavioral features and treatment indicators.

| Variable               | Description                                                                 |
|------------------------|-----------------------------------------------------------------------------|
| `group_id` / `group_label` | Customer tier (Bronze, Silver, Gold, Platinum, VIP)                      |
| `logins_last_week`     | Number of logins in the past 7 days (indicates user activity)              |
| `previous_purchases`   | Number of purchases in the past 30 days (baseline interest)                |
| `viewed_target_category` | Whether the customer viewed items in the target category (binary: 0/1)  |
| `discount_received`    | Whether the customer received a discount (binary treatment: 0/1)           |
| `y`                    | Binary purchase response (1 = purchase, 0 = no purchase)                   |


###  Model Implementation

- Use **PyMC** with **Bayesian inference (NUTS)** for all models.
- M0: shared β coefficients.
- M1: group-level β.
- M2: add interaction term to features.
- M3: add squared term to features.
- Mf: add all term to features.



###  Inference Procedure

- Sample posterior using MCMC.
- Compute posterior predictive probabilities for test set.

---



##  Evaluation Metrics

| Metric | Description |
|--------|-------------|
| **Accuracy** | Test set classification performance |
| **Brier Score** | Measures quality of probabilistic prediction |
| **Log Loss** | Measures quality of probabilistic prediction |

---



##  Interpretation Goals

- Which component provides the largest improvement over the baseline?
- Do some components lead to overconfidence or poor calibration?
- How does Quasi-Bayes inference compare in robustness under each misspecification?

---



In [ ]:
pip install pyro-ppl

In [ ]:
# Import all packages

import numpy as np
import pandas as pd
from scipy.special import expit
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
from scipy.special import expit
import seaborn as sns
import matplotlib.pyplot as plt
import pytensor.tensor as pt
from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss, log_loss
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import arviz as az

import pyro
import pyro.infer.mcmc as pmcmc
from pyro.infer.mcmc import NUTS

# Data simulation


In [ ]:
# Data simulation
# Set random seed for reproducibility
np.random.seed(50)

# Configuration
group_labels = ["Bronze", "Silver", "Gold", "Platinum", "VIP"]
n_groups = len(group_labels)
n_per_group = 10000  # observations per group

# Group-level multiplier: higher-tier customers are more responsive to discounts
group_effects = np.linspace(0.5, 2.0, n_groups)

# Empty list to store simulated rows
rows = []

for j in range(n_groups):
    group_name = group_labels[j]
    group_boost = group_effects[j]

    for _ in range(n_per_group):
        # Features
        logins_last_week = np.random.poisson(lam=5)                   # user activeness
        previous_purchases = np.random.poisson(lam=2)                 # prior purchase behavior
        viewed_target_category = np.random.binomial(1, p=0.5)         # relevance exposure
        discount_received = np.random.binomial(1, p=0.5)              # marketing treatment

        # True logit: combine all meaningful effects
        U = (
            -4.5
            + 0.1 * previous_purchases
            + 0.6 * viewed_target_category
            + 0.9 * discount_received
            + 3 * viewed_target_category * discount_received
            + 1.1 * group_boost * discount_received
            + 0.13 * previous_purchases**2
            + np.random.normal(0, 1)  # heavy-tailed noise
            )

        y = 1 if U >0 else 0

        # Store observation
        rows.append({
            "group_id": j,
            "group_label": group_name,
            "logins_last_week": logins_last_week,
            "previous_purchases": previous_purchases,
            "viewed_target_category": viewed_target_category,
            "discount_received": discount_received,
            "y": y
        })

# Create DataFrame
df_simulated_test = pd.DataFrame(rows)

df_simulated_test.head()

print(df_simulated_test['y'].value_counts())

In [ ]:
def generate_noise(noise_type, size):
    """
    Generate random noise from a specified distribution.

    Args:
        noise_type (str): Type of noise to generate. Supported values are:
            - "normal": Standard normal distribution N(0, 1).
            - "t": Student's t-distribution with 2 degrees of freedom.
            - "cauchy": Standard Cauchy distribution.
            - "contaminated": Contaminated normal distribution.
              Starts from N(0,1) and adds high-variance noise (N(0,5))
              to 10% of randomly selected samples.
        size (int): Number of samples to generate.

    Returns:
        np.ndarray: Array of generated noise samples.

    Raises:
        ValueError: If an unsupported noise_type is provided.
    """
    if noise_type == "normal":
        return np.random.normal(0, 1, size=size)
    elif noise_type == "t":
        return np.random.standard_t(df=2, size=size)
    elif noise_type == "cauchy":
        return np.random.standard_cauchy(size=size)
    elif noise_type == "contaminated":
        base = np.random.normal(0, 1, size=size)
        outlier_idx = np.random.choice(size, size=int(0.1 * size), replace=False)
        base[outlier_idx] += np.random.normal(0, 5, size=len(outlier_idx))
        return base
    else:
        raise ValueError("Unsupported noise type")


def simulate_dataset(noise_type,n_per_group):
    """
    Simulate a synthetic customer-choice dataset with group heterogeneity,
    interactions, nonlinear effects, and noise.

    Args:
        noise_type (str): Type of noise distribution used to generate latent utilities.
            Supported values are the same as in `generate_noise` ("normal", "t",
            "cauchy", "contaminated").
        n_per_group (int): Number of samples to generate per group.

    Returns:
        pd.DataFrame: A standardized dataset with the following columns:
            - "group_id": Numeric ID of the group.
            - "group_label": Name of the group.
            - "logins_last_week": Scaled count of logins in the last week.
            - "previous_purchases": Scaled count of previous purchases.
            - "viewed_target_category": Binary indicator (1 if viewed target category).
            - "discount_received": Binary indicator (1 if discount was received).
            - "y": Binary outcome (1 if latent utility > 0, else 0).

    """
    rows = []
    noise_vector = generate_noise(noise_type, size=n_groups * n_per_group)
    noise_counter = 0

    for j in range(n_groups):
        group_name = group_labels[j]
        group_boost = group_effects[j]

        for _ in range(n_per_group):
            logins_last_week = np.random.poisson(lam=5)
            previous_purchases = np.random.poisson(lam=2)
            viewed_target_category = np.random.binomial(1, p=0.5)
            discount_received = np.random.binomial(1, p=0.5)

            U = (
                -4.5
                + 0.1 * previous_purchases
                + 0.6 * viewed_target_category
                + 0.9 * discount_received
                + 3 * viewed_target_category * discount_received
                + 1.1 * group_boost * discount_received
                + 0.13 * previous_purchases**2
                + np.random.normal(0, 1)
                )
            noise_counter += 1


            y = 1 if U >0 else 0

            rows.append({
                "group_id": j,
                "group_label": group_name,
                "logins_last_week": logins_last_week,
                "previous_purchases": previous_purchases,
                "viewed_target_category": viewed_target_category,
                "discount_received": discount_received,
                "y": y
            })
    df_simulated = pd.DataFrame(rows)
    scaler = StandardScaler()
    df_simulated[["logins_last_week", "previous_purchases"]] = scaler.fit_transform(
    df_simulated[["logins_last_week", "previous_purchases"]])
    return df_simulated


feature_cols = [
    "logins_last_week",
    "previous_purchases",
    "viewed_target_category",
    "discount_received"
]

##  Why Normalize Features in Bayesian Hierarchical Models?

Feature normalization helps ensure **stable inference and fair comparisons** in Bayesian and Quasi-Bayesian models.

---

###  Benefits of Normalization

- **Faster, more stable sampling**  
  NUTS/HMC samplers perform better when all features are on a similar scale.
  
- **Better prior behavior**  
  Priors like `β ~ Normal(0, 1)` assume standardized inputs.
  
- **More interpretable posteriors**  
  Coefficients are easier to compare when features are normalized.
  
- **Consistent across models**  
  Helps maintain numerical stability in both simple and complex structures.

---

###  When to Normalize

| Feature Type             | Normalize? |
|--------------------------|------------|
| Continuous (e.g. counts) |  Yes      |
| Binary or categorical    |  No       |

---

###  How to Normalize

Use **z-score standardization**:

$$
z = \frac{x - \mu}{\sigma}
$$

Apply to:
- `logins_last_week`
- `previous_purchases`

Leave binary features like `viewed_target_category`, `discount_received` unchanged.

---

###  Note

> While normalization may not change **relative model rankings**,  
> it improves **convergence, interpretability, and sampling efficiency** —  
> especially in hierarchical and Quasi-Bayesian models.

In [ ]:
scaler = StandardScaler()
df_simulated_test[["logins_last_week", "previous_purchases"]] = scaler.fit_transform(
    df_simulated_test[["logins_last_week", "previous_purchases"]]
)

feature_cols = [
    "logins_last_week",
    "previous_purchases",
    "viewed_target_category",
    "discount_received"
]

# Scenario 1 : Utility misspecification

In [ ]:
def phi(x):
    """
    Standard normal cumulative distribution function (CDF).

    Args:
        x (float or array-like): Input value(s).

    Returns:
        float or array-like: The probability that a standard normal random
        variable is less than or equal to x.
    """
    return 0.5 * (1 + pt.erf(x / pt.sqrt(2)))


def define_model(df, feature_cols, group_idx=None,
                 group=False, interaction=False, nonlinear=False):
    """
    Construct a PyMC model skeleton (design matrix X, coefficients beta, and linear
    predictor eta) without attaching a likelihood. You can later add either a
    Bayesian likelihood (e.g., probit/logit) or plug in a Quasi-Bayesian
    objective.

    Args:
        df (pd.DataFrame): Input dataset. Must contain columns in `feature_cols`,
            and always "y". If `group=True`, must also contain "group_id".
            If `interaction=True`, must contain
            "viewed_target_category" and "discount_received".
            If `nonlinear=True`, must contain "previous_purchases".
        feature_cols (list[str]): Base feature column names to include in X.
        group_idx (np.ndarray or None): Integer array of length N mapping each row
            to a group index in {0, …, G-1}. Required when `group=True`.
        group (bool): If True, use a hierarchical prior and group-specific betas
            with Normal(mu, sigma) per coefficient.
        interaction (bool): If True, add an extra column
            `interaction = viewed_target_category * discount_received`.
        nonlinear (bool): If True, add an extra column
            `purchases_squared = previous_purchases ** 2`.

    Returns:
        tuple:
            - model (pm.Model): A PyMC model object with data container(s) and priors
              over coefficients but **no likelihood** attached.
            - eta (pt.TensorVariable): Linear predictor of shape (N,).
            - y (np.ndarray): Binary target array extracted from `df["y"]]`.
    """
    X = df[feature_cols].copy()

    # Add interaction term if enabled
    if interaction:
        X["interaction"] = df["viewed_target_category"] * df["discount_received"]

    # Add nonlinear term if enabled
    if nonlinear:
        X["purchases_squared"] = df["previous_purchases"] ** 2  # assume preprocessed if needed

    X_np = X.values.astype("float64")
    y = df["y"].values
    N, K = X.shape

    with pm.Model() as model:
        X_shared = pm.Data("X", X_np)
        if group:
            mu = pm.Normal("mu", mu=0, sigma=1, shape=K)
            sigma = pm.Exponential("sigma", lam=1.0, shape=K)
            beta = pm.Normal("beta", mu=mu, sigma=sigma, shape=(df["group_id"].nunique(), K))
            eta = pt.sum(X_shared * beta[group_idx], axis=1)
        else:
            beta = pm.Normal("beta", mu=0, sigma=1, shape=K)
            eta = pt.dot(X_shared, beta)

    return model, eta, y

In [ ]:
def run_bayesian_model(model, eta, y, link="probit", **sample_kwargs):
    """
    Attach a Bernoulli likelihood to the provided linear predictor and run MCMC
    sampling in PyMC.

    Args:
        model (pm.Model): A PyMC model (typically from `define_model`) that already
            defines the linear predictor `eta` and any priors but has no likelihood.
        eta (pt.TensorVariable): Linear predictor of shape (N,). Typically aano/pytensor
            tensor built from the design matrix and coefficients.
        y (array-like): Binary observations in {0, 1}, length N.
        link (str, optional): Link function to map `eta` to probabilities.
            - "probit": uses Φ(eta) via `phi`.
            - "logit": uses logistic σ(eta).
            Defaults to "probit".
        **sample_kwargs: Extra keyword arguments passed directly to `pm.sample`
            (e.g., draws, tune, target_accept, chains, cores, random_seed).

    Returns:
        arviz.InferenceData: Posterior samples returned by `pm.sample`.

    """
    with model:
      eps = 1e-6

      if link == "probit":
          p_raw = phi(eta)
      elif link == "logit":
          p_raw = pm.math.sigmoid(eta)

      p = pm.Deterministic("p", pm.math.clip(p_raw, eps, 1 - eps))
      pm.Bernoulli("y_obs", p=p, observed=y)
      trace = pm.sample(**sample_kwargs)
    return trace



In [ ]:
def loss_fn(y_true, y_pred, kind,alpha = None):
    """
    Compute different loss functions for binary outcomes.

    Args:
        y_true (array-like): Ground-truth binary labels (0 or 1).
        y_pred (array-like): Predicted probabilities in (0, 1). Values are
            clipped to [1e-6, 1 - 1e-6] for numerical stability.
        kind (str): Type of loss function. Supported options:
            - "bce": Binary cross-entropy loss
                L = - Σ [ y log(p) + (1 - y) log(1 - p) ]
            - "squared": Squared error loss
                L = Σ (y - p)^2
            - "huber": Huber loss (δ = 1.0 fixed)
                Smooth transition between squared error (small residuals)
                and absolute error (large residuals).
            - "sph": Scaled pseudo-Huber loss (requires `alpha` > 0)
                ℓ_SPH,α(t) = α √(1 + α²) ( √(1 + (t/α)²) - 1 )
        alpha (float, optional): Scaling parameter required only for "sph".

    Returns:
        pt.TensorVariable: Scalar tensor representing the total loss
        (sum over all samples).
    """
    p = pt.clip(y_pred, 1e-6, 1 - 1e-6)
    if kind == "bce":
        return -pt.sum(y_true * pt.log(p) + (1 - y_true) *pt.log(1 - p))
    elif kind == "squared":
        return pt.sum(pt.sqr(y_true - p))
    elif kind == "huber":
        delta = 1.0
        residual = y_true - p
        return pt.sum(pt.switch(pt.abs(residual) <= delta,
                                0.5 * residual**2,
                                delta * (pt.abs(residual) - 0.5 * delta)))
    elif kind == "sph":
        t = y_true - p
        scale = alpha * pt.sqrt(1.0 + alpha**2)
        loss_i = scale * (pt.sqrt(1.0 + (t / alpha)**2) - 1.0)
        return pt.sum(loss_i)
    else:
        raise ValueError("Unknown loss kind")

In [ ]:
def run_quasi_model(model, eta, y, loss_kind="bce", **sample_kwargs):
    """
    Attach a loss-based (Quasi-Bayesian) objective to a PyMC model
    and run sampling. The likelihood is replaced by a `Potential` equal to the
    negative loss, i.e., log-posterior ∝ prior − loss(y, p).

    Args:
        model (pm.Model): A PyMC model (e.g., from `define_model`) that defines
            a linear predictor `eta` and priors but has no likelihood attached.
        eta (pt.TensorVariable): Linear predictor of shape (N,).
        y (array-like): Binary targets in {0, 1}, length N.
        loss_kind (str, optional): Loss to use for the quasi objective. One of:
            - "bce": Binary cross-entropy.
            - "squared": Squared error.
            - "huber": Huber (δ = 1).
            - "sph": Scaled pseudo-Huber (introduces α > 0).
        **sample_kwargs: Extra arguments forwarded to `pm.sample`
            (e.g., draws, tune, target_accept, chains, cores, random_seed).

    Returns:
        arviz.InferenceData: Posterior samples under the chosen quasi objective.

    """
    with model:
        p = pm.Deterministic("p", phi(eta))

        step_kwargs = {}
        alpha_sq_rv = None
        alpha_det = None

        if loss_kind == "sph":
            alpha_sq_rv = pm.Gamma("alpha_sq", alpha=1.0, beta=1.0)
            alpha_det = pm.Deterministic("alpha", pt.sqrt(alpha_sq_rv))
            loss = loss_fn(y, p, "sph", alpha=alpha_det)
            step_kwargs["step"] = [pm.Slice(vars=[alpha_sq_rv])]
        else:
            loss = loss_fn(y, p, loss_kind)

        pm.Potential(f"{loss_kind}_loss", -loss)

        trace = pm.sample(**{**step_kwargs, **sample_kwargs})

    return trace

In [ ]:
def run_experiment_loop(
    seeds,
    feature_cols,
    noise_type_for_train="normal",
    npergroup=200,
    group=False, interaction=False, nonlinear=False,
    use_quasi=False, loss_kind="bce",
    draws=1000, tune=1000, target_accept=0.9):
    results = []
    """
    Run repeated simulate–fit–evaluate cycles over multiple random seeds.

    Workflow:
        For each seed:
          1) Simulate a training set (size = n_groups * npergroup) and a large
             test set (fixed at 10,000) from the specified noise type.
          2) Build a PyMC model skeleton via `define_model` with optional
             group heterogeneity, interaction, and nonlinear terms.
          3) Fit either:
               - Bayesian model with Bernoulli likelihood (probit/logit), or
               - Quasi-/Generalized-Bayesian model using a loss via `pm.Potential`.
          4) Compute posterior predictive probabilities on the test set by
             averaging Φ(Xβ) across all posterior β samples (probit link).
          5) Report Accuracy, Log-loss, and Brier score.

    Args:
        seeds (Iterable[int]): Random seeds to iterate over; one experiment per seed.
        feature_cols (list[str]): Base feature names used to build X.
        noise_type_for_train (str, optional): Noise family for simulation
            ("normal", "t", "cauchy", "contaminated"). Defaults to "normal".
        npergroup (int, optional): Number of training samples per group. Defaults to 200.
        group (bool, optional): If True, fit a hierarchical model with group-specific betas.
        interaction (bool, optional): If True, add X[:, "interaction"] =
            viewed_target_category * discount_received.
        nonlinear (bool, optional): If True, add X[:, "purchases_squared"] =
            previous_purchases ** 2.
        use_quasi (bool, optional): If True, fit the quasi model; otherwise fit the
            standard Bayesian likelihood model.
        loss_kind (str, optional): Loss name when `use_quasi=True`
            ("bce", "squared", "huber", "sph"). Ignored for Bayesian mode.
        draws (int, optional): Number of MCMC draws for `pm.sample`. Defaults to 1000.
        tune (int, optional): Number of tuning steps for `pm.sample`. Defaults to 1000.
        target_accept (float, optional): Target acceptance rate for NUTS. Defaults to 0.9.

    Returns:
        pd.DataFrame: One row per seed with columns:
            - "seed": Seed value used.
            - "acc": Accuracy on the test set using threshold 0.5 on mean p.
            - "logloss": Log-loss computed on mean posterior predictive probs.
            - "brier": Brier score on mean posterior predictive probs.
            - "model_type": "quasi" or "bayes".
            - "loss_kind": Loss name when quasi is used; otherwise None.

    """

    for seed in seeds:
        np.random.seed(seed)

        df_train = simulate_dataset(noise_type_for_train,npergroup)
        group_idx_train = df_train["group_id"].values if group else None

        df_test = simulate_dataset(noise_type_for_train,10000)
        X_test = df_test[feature_cols].copy()
        y_test = df_test["y"].values
        group_idx_test = df_test["group_id"].values if group else None
        if interaction:
          X_test["interaction"] = X_test["viewed_target_category"] * X_test["discount_received"]
        if nonlinear:
          X_test["purchases_squared"] = X_test["previous_purchases"] ** 2
        X_test = X_test.values.astype("float64")


        model, eta, y = define_model(
            df_train, feature_cols,
            group_idx=group_idx_train,
            group=group, interaction=interaction, nonlinear=nonlinear
        )

        if use_quasi:
            trace = run_quasi_model(
                model, eta, y,
                loss_kind=loss_kind,
                draws=draws, tune=tune, target_accept=target_accept,
                return_inferencedata=True,     idata_kwargs={"log_likelihood": True}
            )
        else:
            trace = run_bayesian_model(
                model, eta, y,
                draws=draws, tune=tune, target_accept=target_accept,
                return_inferencedata=True,     idata_kwargs={"log_likelihood": True}
            )


        beta_da = trace.posterior["beta"].stack(sample=("chain", "draw"))

        if beta_da.ndim == 2:
          beta = beta_da.transpose("sample", ...).values

          eta = X_test @ beta.T

        else:
          beta = beta_da.transpose("sample", ...).values
          S, G, K = beta.shape


          beta_g = beta[:, group_idx_test, :]

          eta = np.einsum("snk,nk->ns", beta_g, X_test)

        p_samples = norm.cdf(eta)


        p_mean = p_samples.mean(axis=1)

        y_pred = (p_mean >= 0.5).astype(int)

        acc = accuracy_score(y_test, y_pred)
        logloss_val = log_loss(y_test, p_mean, labels=[0,1])
        brier = brier_score_loss(y_test, p_mean)

        results.append({
            "seed": seed,
            "acc": acc,
            "logloss": logloss_val,
            "brier": brier,
            "model_type": "quasi" if use_quasi else "bayes",
            "loss_kind": loss_kind if use_quasi else None
        })

    return pd.DataFrame(results)

## 1. Classical Bayesian model

In [ ]:
df_results_bayes_m0 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=False,
    nonlinear=False,
    use_quasi=False
)

df_results_bayes_m1 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=True,
    interaction=False,
    nonlinear=False,
    use_quasi=False
)

df_results_bayes_m2 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=True,
    nonlinear=False,
    use_quasi=False
)

df_results_bayes_m3 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=False,
    nonlinear=True,
    use_quasi=False
)
df_results_bayes_mf = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=True,
    interaction=True,
    nonlinear=True,
    use_quasi=False
)



In [ ]:
print(df_results_bayes_m0)
print(df_results_bayes_m1)
print(df_results_bayes_m2)
print(df_results_bayes_m3)
print(df_results_bayes_mf)

## 2. Quasi Bayes model

In [ ]:
df_results_quasi_m0 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=False,
    nonlinear=False,
    use_quasi=True
)

print(df_results_quasi_m0)

In [ ]:
df_results_quasi_m1 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=True,
    interaction=False,
    nonlinear=False,
    use_quasi=True
)

print(df_results_quasi_m1)

In [ ]:
df_results_quasi_m2 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=True,
    nonlinear=False,
    use_quasi=True
)

print(df_results_quasi_m2)

In [ ]:
df_results_quasi_m3 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=False,
    nonlinear=True,
    use_quasi=True
)
print(df_results_quasi_m3)

In [ ]:
df_results_quasi_mf = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=True,
    interaction=True,
    nonlinear=True,
    use_quasi=True
)

print(df_results_quasi_mf)


## 3. SPH Bayes model

In [ ]:
df_results_sph_m0 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=False,
    nonlinear=False,
    use_quasi=True,loss_kind = "sph"
)

print(df_results_sph_m0)

In [ ]:
df_results_sph_m1 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=True,
    interaction=False,
    nonlinear=False,
    use_quasi=True,loss_kind = "sph"
)

print(df_results_sph_m1)

In [ ]:
df_results_sph_m2 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=True,
    nonlinear=False,
    use_quasi=True,loss_kind = "sph"
)

print(df_results_sph_m2)

In [ ]:
df_results_sph_m3 = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=False,
    interaction=False,
    nonlinear=True,
    use_quasi=True,loss_kind = "sph"
)
print(df_results_sph_m3)

In [ ]:
df_results_sph_mf = run_experiment_loop(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
    group=True,
    interaction=True,
    nonlinear=True,
    use_quasi=True,loss_kind = "sph"
)

print(df_results_sph_mf)

## 4. KSD Bayes model

In [ ]:
import torch
import pandas as pd
import numpy as np
from typing import Tuple, Optional, Dict

def build_features(
    df: pd.DataFrame,
    feature_cols,
    interaction: bool,
    nonlinear: bool,
    group: bool
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, int, int]:
    """
    Build PyTorch-ready feature tensors (with bias), target, group indices, and
    optional engineered features (interaction, nonlinear) from a pandas DataFrame.

    Args:
        df (pd.DataFrame): Input data containing at least `feature_cols`, "y",
            and, if `group=True`, "group_id". For engineered features:
            - If `interaction=True`, expects "viewed_target_category" and "discount_received".
            - If `nonlinear=True`, expects "previous_purchases".
        feature_cols (Iterable[str]): Base feature names to include. A constant
            bias term (column of 1.0) is appended automatically.
        interaction (bool): If True, compute an extra column:
            (viewed_target_category * discount_received).
        nonlinear (bool): If True, compute an extra column:
            (previous_purchases ** 2).
        group (bool): If True, return integer group codes derived from "group_id"
            and set G to the number of unique groups; otherwise a zero vector and G=1.

    Returns:
        Tuple[
            torch.Tensor,  # X: (N, D) float32, base features + bias
            torch.Tensor,  # y: (N,) float32, binary targets in {0., 1.}
            torch.Tensor,  # group_ids: (N,) int64 (long), group codes (or zeros if group=False)
            torch.Tensor,  # extra: (N, 2) float32, [interaction, nonlinear] (zeros for disabled parts)
            int,           # D: number of columns in X (including bias)
            int            # G: number of groups (>=1)
        ]

    """
    X_df = df[list(feature_cols)].copy()
    X_df["bias"] = 1.0
    X = torch.tensor(X_df.values, dtype=torch.float32)

    y = torch.tensor(df["y"].values, dtype=torch.float32)
    if group:
        gids_raw = pd.Categorical(df["group_id"])
        group_ids = torch.tensor(gids_raw.codes, dtype=torch.long)
        G = len(gids_raw.categories)
    else:
        group_ids = torch.zeros(len(df), dtype=torch.long)
        G = 1

    inter = torch.tensor(
        (df["viewed_target_category"] * df["discount_received"]).values,
        dtype=torch.float32
    ).unsqueeze(1) if interaction else torch.zeros((len(df), 1), dtype=torch.float32)

    nonlin = torch.tensor(
        (df["previous_purchases"] ** 2).values,
        dtype=torch.float32
    ).unsqueeze(1) if nonlinear else torch.zeros((len(df), 1), dtype=torch.float32)

    extra = torch.cat([inter, nonlin], dim=1)
    D = X.shape[1]
    return X, y, group_ids, extra, D, G


def unpack_theta(theta: torch.Tensor, D: int, G: int, use_gamma1: bool, use_gamma2: bool):
    """
    Unpack a flat parameter vector into structured components:
    group-specific betas and optional scalars (gamma1, gamma2).

    Args:
        theta (torch.Tensor): Flat parameter vector (shape: (G*D [+ 1 if use_gamma1] [+ 1 if use_gamma2],)).
        D (int): Number of base features in X (including bias if present).
        G (int): Number of groups (>=1).
        use_gamma1 (bool): Whether to include and return gamma1 (interaction weight).
        use_gamma2 (bool): Whether to include and return gamma2 (nonlinear weight).

    Returns:
        dict: {
            "betas":  torch.Tensor of shape (G, D),
            "gamma1": scalar tensor (0.0 if not used),
            "gamma2": scalar tensor (0.0 if not used),
        }
    """
    idx = 0
    if G > 1:
        betas = theta[idx: idx + G * D].reshape(G, D)
        idx += G * D
    else:
        betas = theta[idx: idx + D].unsqueeze(0)
        idx += D

    gamma1 = theta[idx] if use_gamma1 else torch.tensor(0.0, dtype=theta.dtype, device=theta.device)
    idx += 1 if use_gamma1 else 0

    gamma2 = theta[idx] if use_gamma2 else torch.tensor(0.0, dtype=theta.dtype, device=theta.device)
    idx += 1 if use_gamma2 else 0

    return {"betas": betas, "gamma1": gamma1, "gamma2": gamma2}


def eta_fn(theta: torch.Tensor, X: torch.Tensor, group_ids: torch.Tensor,
           extra: torch.Tensor, D: int, G: int,
           use_gamma1: bool, use_gamma2: bool) -> torch.Tensor:
    """
    Compute the linear predictor η(x) for a (possibly) group-heterogeneous model:
        η_i = x_i^T β_{g[i]} + γ1 * interaction_i + γ2 * nonlinear_i

    Args:
        theta (torch.Tensor): Flat parameter vector compatible with `unpack_theta`.
        X (torch.Tensor): Design matrix of shape (N, D) matching the betas.
        group_ids (torch.Tensor): Long tensor of shape (N,) with entries in {0, …, G-1}.
        extra (torch.Tensor): Tensor of shape (N, 2) where:
            - extra[:, 0] = interaction feature (or zeros if disabled),
            - extra[:, 1] = nonlinear feature (or zeros if disabled).
        D (int): Number of base features (columns of X).
        G (int): Number of groups.
        use_gamma1 (bool): If True, include γ1 * extra[:, 0].
        use_gamma2 (bool): If True, include γ2 * extra[:, 1].

    Returns:
        torch.Tensor: Linear predictor η of shape (N,), dtype/device following `X`.

    """
    pars = unpack_theta(theta, D, G, use_gamma1, use_gamma2)
    betas = pars["betas"]
    xb = (X * betas[group_ids]).sum(dim=1)
    eta = xb
    if use_gamma1:
        eta = eta + pars["gamma1"] * extra[:, 0]
    if use_gamma2:
        eta = eta + pars["gamma2"] * extra[:, 1]
    return eta


SQRT2 = 1.4142135623730951
def norm_cdf(z: torch.Tensor) -> torch.Tensor:
    """
    Standard normal cumulative distribution function Φ(z) applied elementwise.

    Args:
        z (torch.Tensor): Input tensor.

    Returns:
        torch.Tensor: Elementwise CDF values in (0, 1).
    """
    return 0.5 * (1.0 + torch.erf(z / SQRT2))

def norm_ppf(u: torch.Tensor,eps: float = 1e-6) -> torch.Tensor:
    """
    Standard normal percent-point function (inverse CDF) Φ^{-1}(u) applied elementwise.

    Args:
        u (torch.Tensor): Probabilities. Values are clamped to [eps, 1 - eps]
            for numerical stability.
        eps (float, optional): Clamp margin to avoid infinities. Defaults to 1e-6.

    Returns:
        torch.Tensor: Elementwise quantiles.
    """
    u = u.clamp(eps, 1.0 - eps)
    return SQRT2 * torch.erfinv(2.0 * u - 1.0)

@torch.no_grad()
def sample_truncnorm_from_probit(eta: torch.Tensor, y: torch.Tensor,eps: float = 1e-6) -> torch.Tensor:
    """
    Draw latent Gaussian utilities z from the probit data augmentation
    (Albert–Chib) scheme:
        z | (y=1, η) ~ N(η, 1) truncated to (0, ∞)
        z | (y=0, η) ~ N(η, 1) truncated to (-∞, 0]

    Uses inverse-transform sampling with numerically stable clamping.

    Args:
        eta (torch.Tensor): Linear predictor η of shape (N,).
        y (torch.Tensor): Binary outcomes in {0, 1} (shape (N,)). Values > 0.5
            are treated as 1.
        eps (float, optional): Clamp margin for CDF/PPF to avoid 0/1. Defaults to 1e-6.

    Returns:
        torch.Tensor: Latent samples z of shape (N,).
    """
    N = eta.shape[0]
    a = (0.0 - eta)
    Phi_a = norm_cdf(a).clamp(eps, 1.0 - eps)

    u1 = Phi_a + (1.0 - Phi_a) * torch.rand(N, device=eta.device)
    z1 = eta + norm_ppf(u1)

    u0 = torch.rand(N, device=eta.device) * Phi_a
    z0 = eta + norm_ppf(u0)

    return torch.where(y > 0.5, z1, z0)

def _poly_stats(r: torch.Tensor, gamma: float):
    """
    Utility for polynomial kernel derivatives with respect to r = ||x - x'||^2 (or a scalar proxy).
    Args:
        r (torch.Tensor): Nonnegative tensor (often pairwise squared distances).
        gamma (float): Polynomial kernel exponent γ.

    Returns:
        Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """

    k   = (1.0 + r).pow(gamma)
    kp  = gamma * (1.0 + r).pow(gamma - 1.0)
    kpp = gamma * (gamma - 1.0) * (1.0 + r).pow(gamma - 2.0)
    return k, kp, kpp

def imq_cache_from_u(u: torch.Tensor, c: float = None, beta: float = 0.5):
    """
    Build cached tensors for the IMQ (Inverse Multiquadric) kernel and its
    r-derivatives for a vector of scalars u (e.g., latent utilities).

    Args:
        u (torch.Tensor): 1D or (N,1) tensor; will be reshaped to (N,1).
        c (float, optional): IMQ scale parameter. If None, uses the median
            heuristic on pairwise distances of u (once per call).
        beta (float, optional): IMQ exponent β. Defaults to 0.5.

    Returns:
        dict: Cached components:
            - "k":      (N,N) kernel matrix (A^(-β))
            - "kp":     (N,N) first derivative wrt A ( -β A^(-β-1) )
            - "dA_du":  (N,N) ∂A/∂u   =  2 (u - u')
            - "dA_dup": (N,N) ∂A/∂u'  = -2 (u - u')
            - "trace_base": (N,N) base term for mixed second derivatives:
                  k'' * (∂A/∂u)(∂A/∂u') + k' * (∂²A/∂u∂u')
                  where ∂²A/∂u∂u' = -2
            - "ones_col": (N,1) column of ones (utility for reductions)
            - "ones_row": (1,N) row of ones
            - "c": chosen scale (float)
            - "beta": IMQ exponent (float)
    """
    u = u.view(-1, 1)
    device = u.device
    N = u.shape[0]
    diff = u - u.t()
    r2 = diff.pow(2)

    if c is None:
        tri = r2[torch.triu(torch.ones_like(r2, dtype=torch.bool), diagonal=1)]
        med = torch.sqrt(tri.median()).clamp_min(1e-8)
        c = med

    A   = c*c + r2
    k   = A.pow(-beta)
    kp  = -beta * A.pow(-beta - 1.0)
    kpp = (-beta) * (-beta - 1.0) * A.pow(-beta - 2.0)

    dA_du  =  2.0 * diff
    dA_dup = -2.0 * diff

    trace_base = kpp * dA_du * dA_dup + kp * (-2.0)

    ones_col = torch.ones(N, 1, device=device)
    ones_row = torch.ones(1, N, device=device)

    return {
        "k": k, "kp": kp,
        "dA_du": dA_du, "dA_dup": dA_dup,
        "trace_base": trace_base,
        "ones_col": ones_col, "ones_row": ones_row,
        "c": c, "beta": beta
    }

def ksd2_vstat_imq_cached(u: torch.Tensor, eta: torch.Tensor, cache: dict) -> torch.Tensor:
    """
    Compute the V-statistic estimator of the squared Kernelized Stein Discrepancy (KSD^2)
    using a cached IMQ kernel and its derivatives.

    Args:
        u (torch.Tensor): Latent samples (N,) or (N,1).
        eta (torch.Tensor): Linear predictor / score mean (N,) or (N,1).
        cache (dict): Precomputed kernel components from `imq_cache_from_u`, including:
            - "k": (N,N) kernel matrix
            - "kp": (N,N) derivative wrt A
            - "dA_du": (N,N) ∂A/∂u
            - "dA_dup": (N,N) ∂A/∂u'
            - "trace_base": (N,N) cross second derivative base term
            - "ones_col", "ones_row": helper tensors for broadcasting

    Returns:
        torch.Tensor: Scalar tensor with the V-statistic estimate of KSD^2.
    """
    u   = u.view(-1, 1)
    eta = eta.view(-1, 1)
    N   = u.shape[0]
    k         = cache["k"]
    dA_du     = cache["dA_du"]
    dA_dup    = cache["dA_dup"]
    trace_base= cache["trace_base"]
    ones_col  = cache["ones_col"]
    ones_row  = cache["ones_row"]


    s = -(u - eta)
    s_dot = s @ s.t()


    kp = cache["kp"]
    grad_u_k  = kp * dA_du
    grad_up_k = kp * dA_dup

    term1 = k * s_dot
    term2 = (s @ ones_row) * grad_up_k
    term3 = (ones_col @ s.t()) * grad_u_k
    Umat = term1 + term2 + term3 + trace_base
    return Umat.mean()

def logpost_ksd_bayes(
    theta: torch.Tensor,
    X: torch.Tensor, y: torch.Tensor, group_ids: torch.Tensor,
    extra: torch.Tensor,
    D: int, G: int,
    use_gamma1: bool, use_gamma2: bool,
    u_imputed: torch.Tensor,
    beta_lr: float = 1.0,
    prior_std: float = 1.0,
    cache = None
) -> torch.Tensor:
    """
    Log-posterior for KSD-Bayes with IMQ kernel.

    Objective:
        log p(θ | data) ∝ log prior(θ) − β_lr * N * KSD²(u, η; θ)

    Args:
        theta (torch.Tensor): Flat parameter vector (includes betas, gamma1/2).
        X (torch.Tensor): Feature matrix (N, D).
        y (torch.Tensor): Binary labels (N,). Not directly used here but kept
            for API consistency.
        group_ids (torch.Tensor): Group index per sample (N,).
        extra (torch.Tensor): Extra engineered features (interaction, nonlinear),
            shape (N, 2).
        D (int): Number of features in X.
        G (int): Number of groups.
        use_gamma1 (bool): Whether gamma1 is active in eta_fn.
        use_gamma2 (bool): Whether gamma2 is active in eta_fn.
        u_imputed (torch.Tensor): Latent variables sampled from truncated normals,
            used in Stein discrepancy.
        beta_lr (float, optional): Scaling factor (learning-rate like). Default = 1.0.
        prior_std (float, optional): Std dev of Normal(0, prior_std²) prior on θ.
            Default = 1.0.
        cache (dict, optional): Precomputed IMQ kernel cache from
            `imq_cache_from_u(u_imputed)`.

    Returns:
        torch.Tensor: Scalar log-posterior (float32/float64, depending on inputs).
    """
    eta = eta_fn(theta, X, group_ids, extra, D, G, use_gamma1, use_gamma2)
    ksd2 = ksd2_vstat_imq_cached(u_imputed, eta, cache)
    log_prior = -0.5 * theta.pow(2).sum() / (prior_std**2)
    N = X.shape[0]
    return log_prior - beta_lr * N * ksd2



# @torch.no_grad()

def make_potential_fn(
    X, y, group_ids, extra, D, G, use_gamma1, use_gamma2,
    u_imputed, beta_lr, prior_std, cache
):
    """
    Wrap the KSD-Bayes log-posterior into a potential function compatible with
    gradient-free NUTS implementations (expects a dict with key "theta").

    Returns:
        Callable[[dict], torch.Tensor]: A function that takes {"theta": θ}
        and returns the negative log-posterior (i.e., a potential/energy).
    """
    def _potential(theta_dict):
        theta = theta_dict["theta"]
        return -logpost_ksd_bayes(
            theta, X, y, group_ids, extra, D, G,
            use_gamma1, use_gamma2, u_imputed,
            beta_lr=beta_lr, prior_std=prior_std, cache=cache
        )
    return _potential

def nuts_sample_theta(
    theta_init: torch.Tensor,
    X: torch.Tensor, y: torch.Tensor, group_ids: torch.Tensor, extra: torch.Tensor,
    D: int, G: int, use_gamma1: bool, use_gamma2: bool,
    u_imputed: torch.Tensor, cache: dict,
    beta_lr: float = 1.0, prior_std: float = 1.0,
    num_warmup: int = 300, num_samples: int = 100,
    target_accept_prob: float = 0.8, max_tree_depth: int = 10,
) -> torch.Tensor:
    """
    Run NUTS over θ using a potential defined by KSD-Bayes (negative log-posterior).

    Args:
        theta_init (torch.Tensor): Initial parameter vector θ (flat).
        X, y, group_ids, extra, D, G, use_gamma1, use_gamma2:
            Model/data components passed through to `logpost_ksd_bayes`.
        u_imputed (torch.Tensor): Latent draws used inside the KSD term.
        cache (dict): IMQ kernel cache from `imq_cache_from_u(u_imputed)`.
        beta_lr (float): KSD scaling factor.
        prior_std (float): Std for N(0, prior_std²) prior on θ.
        num_warmup (int): NUTS warmup (adaptation) steps.
        num_samples (int): Number of post-warmup samples to draw.
        target_accept_prob (float): Target acceptance probability for NUTS.
        max_tree_depth (int): Maximum tree depth for NUTS.

    Returns:
        torch.Tensor: Samples of θ with shape (num_samples, dim_theta) on the same
        device as `theta_init`.
    """
    device = theta_init.device
    potential_fn = make_potential_fn(
        X, y, group_ids, extra, D, G, use_gamma1, use_gamma2,
        u_imputed, beta_lr, prior_std, cache
    )

    kernel = NUTS(
        potential_fn=potential_fn,
        target_accept_prob=target_accept_prob,
        max_tree_depth=max_tree_depth,
        adapt_step_size=True,
        adapt_mass_matrix=True,
    )


    mcmc = None
    try:
        mcmc = pmcmc.MCMC(
            kernel,
            num_samples=num_samples,
            warmup_steps=num_warmup,
            initial_params={"theta": theta_init.detach()},
        )
        mcmc.run()
    except TypeError:
        mcmc = pmcmc.MCMC(
            kernel,
            num_samples=num_samples,
            initial_params={"theta": theta_init.detach()},
        )
        mcmc.run(num_warmup=num_warmup)

    samples = mcmc.get_samples(group_by_chain=False)["theta"].to(device)
    return samples

def init_theta(D: int, G: int, use_gamma1: bool, use_gamma2: bool, scale: float = 0.1) -> torch.Tensor:
    """
    Initialize a flat parameter vector θ for the (possibly) hierarchical model.

    Layout size:
        - If G > 1: G * D coefficients (group-specific betas)
        - Else:     D coefficients (single beta)
        - +1 if use_gamma1 (interaction coefficient γ1)
        - +1 if use_gamma2 (nonlinear  coefficient γ2)

    Args:
        D (int): Number of base features (including bias if present).
        G (int): Number of groups (>=1).
        use_gamma1 (bool): Include γ1 parameter if True.
        use_gamma2 (bool): Include γ2 parameter if True.
        scale (float, optional): Std dev for N(0, scale²) init. Defaults to 0.1.

    Returns:
        torch.Tensor: θ ∈ R^k with k computed from the layout above.
    """
    k = (G * D if G > 1 else D) + (1 if use_gamma1 else 0) + (1 if use_gamma2 else 0)
    return scale * torch.randn(k)

def predict_probit(
    theta_mean: torch.Tensor,
    df_test: pd.DataFrame,
    feature_cols,
    interaction: bool, nonlinear: bool, group: bool
) -> Tuple[torch.Tensor, Dict[str, float]]:
    """
    Posterior-predictive probabilities and metrics under a probit link using a
    single parameter vector (e.g., posterior mean of θ).

    Steps:
      1) Build tensors from `df_test` (adds bias; optionally interaction/nonlinear).
      2) Compute η = xᵢᵀ β_{g[i]} + γ1 * interaction + γ2 * nonlinear.
      3) Map via probit: p = Φ(η).
      4) Return p and metrics (accuracy, Brier, log loss).

    Args:
        theta_mean (torch.Tensor): Flat parameter vector θ (often posterior mean).
        df_test (pd.DataFrame): Test data.
        feature_cols (Iterable[str]): Base feature columns used in X.
        interaction (bool): If True, include interaction feature and γ1 term.
        nonlinear (bool): If True, include squared feature and γ2 term.
        group (bool): If True, use group-specific betas.

    Returns:
        Tuple[torch.Tensor, Dict[str, float]]:
            - p: (N,) tensor of predicted probabilities.
            - metrics: {"accuracy": float, "brier": float, "logloss": float}.
    """
    X, y, group_ids, extra, D, G = build_features(df_test, feature_cols, interaction, nonlinear, group)

    device = theta_mean.device
    X = X.to(device)
    y = y.to(device)
    group_ids = group_ids.to(device)
    extra = extra.to(device)
    eta = eta_fn(theta_mean, X, group_ids, extra, D, G, interaction, nonlinear)
    p = 0.5 * (1.0 + torch.erf(eta / 1.4142135623730951))
    p = p.clamp(1e-8, 1-1e-8)
    yhat = (p > 0.5).float()

    acc = (yhat == y).float().mean().item()
    brier = torch.mean((p - y) ** 2).item()
    logloss = -torch.mean(y * torch.log(p) + (1 - y) * torch.log(1 - p)).item()

    return p, {"accuracy": acc, "brier": brier, "logloss": logloss}


In [ ]:
import numpy as np
import torch
from collections import deque

def fit_ksd_bayes_nuts_ema_ensemble(
    df_train, df_test, feature_cols,
    interaction=True, nonlinear=True, group=True,
    n_outer=60,
    nuts_warmup=300, nuts_samples=30,
    frac_pp_mean=0.5,
    beta_lr=0.01, prior_std=1.0, imq_beta=0.5,
    target_accept_prob=0.90, max_tree_depth=10,
    alpha=0.25,
    K_ens=8,
    K_ma=7,
    tol_logloss=0.01, tol_brier=0.005, tol_acc=0.005,
    device="cuda", verbose=True
):
    """
    Fit KSD-Bayes parameters with an outer-loop EMA + inner NUTS sampler and
    ensemble posterior-predictive smoothing.

    Procedure (per outer iteration t):
      1) Build tensors from `df_train` (optionally with interaction/nonlinear/group).
      2) Compute η = η(θ; X, group_ids, extra).
      3) Draw latent utilities u ~ TruncNormal(η, 1) using probit augmentation.
      4) Build IMQ kernel cache from u (median-heuristic scale).
      5) Run NUTS over θ with potential −logpost_ksd_bayes(θ; u, …) for
         `nuts_warmup` + `nuts_samples`.
      6) Compute inner mean θ̄ (over the `nuts_samples` draws) and update
         θ ← (1−α)·θ + α·θ̄   (EMA).
      7) Form p_mean_this_outer by averaging predictions over the last
         `frac_pp_mean` fraction of θ draws; push to a size-`K_ens` deque and
         compute ensemble p_ens as the mean across the deque.
      8) Track train metrics (accuracy, Brier, logloss). If the moving-average
         improvement over the last `K_ma` rounds is below tolerances
         (`tol_logloss`, `tol_brier`, `tol_acc`) for all three metrics, stop early.

    After the loop:
      - Average predictions over **all** draws in the last NUTS run on `df_test`
        to produce test metrics.

    Args:
        df_train (pd.DataFrame): Training data.
        df_test (pd.DataFrame): Test data.
        feature_cols (Iterable[str]): Base feature columns used to build X.
        interaction (bool): Include interaction feature and learn γ1.
        nonlinear (bool): Include squared feature and learn γ2.
        group (bool): Use group-specific betas (hierarchical).
        n_outer (int): Number of outer EMA iterations (max; may early-stop).
        nuts_warmup (int): NUTS warmup steps per outer iteration.
        nuts_samples (int): NUTS post-warmup samples per outer iteration.
        frac_pp_mean (float): Fraction in (0,1] of most recent θ draws used to
            compute per-outer posterior-predictive mean p.
        beta_lr (float): Scale β in the KSD-Bayes objective.
        prior_std (float): Std for N(0, prior_std²) prior on θ.
        imq_beta (float): IMQ kernel exponent β (>0).
        target_accept_prob (float): NUTS target acceptance probability.
        max_tree_depth (int): NUTS max tree depth.
        alpha (float): EMA mixing rate in (0,1].
        K_ens (int): Ensemble window size for p_ens over outer iterations.
        K_ma (int): Moving-average window for early-stopping comparisons.
        tol_logloss (float): Relative improvement threshold for logloss.
        tol_brier (float): Relative improvement threshold for Brier.
        tol_acc (float): Absolute improvement threshold for accuracy.
        device (str): Torch device for tensors ("cuda" or "cpu").
        verbose (bool): If True, print per-outer training metrics and early-stop info.

    Returns:
        dict:
            - "theta_path": torch.Tensor, shape (T_eff, dim_θ)
                EMA θ snapshots over outer iterations (T_eff ≤ n_outer).
            - "history_train": dict with lists for "logloss", "brier", "accuracy".
            - "stopped_at": int or None, outer iteration index where early-stop occurred.
            - "final_theta": torch.Tensor, last EMA θ (on CPU).
            - "metrics_test": dict with {"accuracy", "brier", "logloss"} computed by
                averaging predictions over all θ draws from the final NUTS run.
    """

    X, y, group_ids, extra, D, G = build_features(df_train, feature_cols, interaction, nonlinear, group)
    X, y, group_ids, extra = X.to(device), y.to(device), group_ids.to(device), extra.to(device)
    use_gamma1, use_gamma2 = interaction, nonlinear


    theta = init_theta(D, G, use_gamma1, use_gamma2).to(device)
    theta_path = []
    history = {"logloss": [], "brier": [], "accuracy": []}
    last_theta_draws = None


    bag = deque(maxlen=K_ens)

    stopped_at = None
    eps = 1e-8

    for t in range(n_outer):

        eta = eta_fn(theta, X, group_ids, extra, D, G, use_gamma1, use_gamma2)
        u = sample_truncnorm_from_probit(eta, y).to(device)


        cache = imq_cache_from_u(u, c=None, beta=imq_beta)


        theta_draws = nuts_sample_theta(
            theta_init=theta,
            X=X, y=y, group_ids=group_ids, extra=extra,
            D=D, G=G, use_gamma1=use_gamma1, use_gamma2=use_gamma2,
            u_imputed=u, cache=cache,
            beta_lr=beta_lr, prior_std=prior_std,
            num_warmup=nuts_warmup, num_samples=nuts_samples,
            target_accept_prob=target_accept_prob, max_tree_depth=max_tree_depth,
        )
        last_theta_draws = theta_draws.detach().cpu()

        theta_inner_mean = theta_draws.mean(0).detach()
        if t == 0:
            theta = theta_inner_mean
        else:
            theta = (1.0 - alpha) * theta + alpha * theta_inner_mean

        theta_path.append(theta.detach().cpu())

        k_pp = max(1, int(nuts_samples * frac_pp_mean))
        ps = []
        for th in theta_draws[-k_pp:]:
            p_th, _ = predict_probit(th, df_train, feature_cols, interaction, nonlinear, group)
            ps.append(p_th.to(device))
        p_mean_this_outer = torch.stack(ps).mean(0)

        bag.append(p_mean_this_outer.detach())
        p_ens = torch.stack(list(bag)).mean(0)
        y_tr = torch.tensor(df_train["y"].values, dtype=torch.float32, device=device)
        acc = ((p_ens > 0.5).float() == y_tr).float().mean().item()
        brier = torch.mean((p_ens - y_tr) ** 2).item()
        logloss = (-y_tr * torch.log(p_ens.clamp(eps, 1-eps))
                   - (1 - y_tr) * torch.log((1 - p_ens).clamp(eps, 1-eps))).mean().item()

        history["logloss"].append(logloss)
        history["brier"].append(brier)
        history["accuracy"].append(acc)

        if verbose:
            print(f"[outer {t:03d}] TRAIN (EMA+K-ens) ll={logloss:.4f}  br={brier:.4f}  acc={acc:.4f}")

        if len(history["logloss"]) >= 2*K_ma:
            def ma_tail(vals, K): return float(np.mean(vals[-K:]))
            ll_r = ma_tail(history["logloss"], K_ma)
            ll_p = ma_tail(history["logloss"][:-K_ma], K_ma)
            br_r = ma_tail(history["brier"],   K_ma)
            br_p = ma_tail(history["brier"][:-K_ma],   K_ma)
            ac_r = ma_tail(history["accuracy"], K_ma)
            ac_p = ma_tail(history["accuracy"][:-K_ma], K_ma)

            ll_impr = (ll_p - ll_r) / max(ll_p, 1e-12)
            br_impr = (br_p - br_r) / max(br_p, 1e-12)
            ac_impr = (ac_r - ac_p)

            stop_ll  = (0 <= ll_impr < tol_logloss)
            stop_br  = (0 <= br_impr < tol_brier)
            stop_acc = (0 <= ac_impr < tol_acc)

            if stop_ll and stop_br and stop_acc:
                stopped_at = t
                if verbose:
                    print(f"[Early stop @ outer {t}] "
                          f"Δll={ll_impr:.3%}, Δbr={br_impr:.3%}, Δacc={ac_impr:.3f}")
                break
    ps = []
    with torch.no_grad():
      for th in last_theta_draws:
        p_th, _ = predict_probit(th.to(device), df_test, feature_cols, interaction, nonlinear, group)
        ps.append(p_th.to(device))

    p_test = torch.stack(ps).mean(0)

    y_te = torch.tensor(df_test["y"].values, dtype=torch.float32, device=device)
    acc_te = ((p_test > 0.5).float() == y_te).float().mean().item()
    brier_te = torch.mean((p_test - y_te) ** 2).item()
    logloss_te = (-y_te * torch.log(p_test.clamp(eps, 1 - eps))
              - (1 - y_te) * torch.log((1 - p_test).clamp(eps, 1 - eps))).mean().item()


    return {
        "theta_path": torch.stack(theta_path),
        "history_train": history,
        "stopped_at": stopped_at,
        "final_theta": theta.detach().cpu(),
       "metrics_test": {
        "accuracy": acc_te,
        "brier": brier_te,
        "logloss": logloss_te
    }
    }


In [ ]:
all_metrics = []
noise_type = "normal"
for seed in range(10):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type=noise_type,
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = noise_type,
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=False, nonlinear=False, group=False,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )
    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)

In [ ]:
all_metrics = []
noise_type="normal"
for seed in range(10):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type=noise_type,
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = noise_type,
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=False, nonlinear=False, group=True,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )

    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)

In [ ]:
all_metrics = []

for seed in range(10):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type="normal",
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = "normal",
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=True, nonlinear=False, group=False,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )
    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)

In [ ]:
all_metrics = []
noise_type = "normal"
for seed in range(10):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type=noise_type,
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = noise_type,
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=False, nonlinear=True, group=False,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )
    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)

In [ ]:
all_metrics = []
noise_type = "normal"
for seed in range(10):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type=noise_type,
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = noise_type,
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=True, nonlinear=True, group=True,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )
    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)

# Scenario 2 : Misspecified error distribution

In [ ]:
def generate_noise(noise_type, size):
    if noise_type == "normal":
        return np.random.normal(0, 1, size=size)

    elif noise_type == "t":
        return np.random.standard_t(df=2.2, size=size) * 3.0

    elif noise_type == "cauchy":
        return np.random.standard_cauchy(size=size) * 4.0

    elif noise_type == "contaminated":
        base = np.random.normal(0, 1, size=size)
        outlier_idx = np.random.choice(size, size=int(0.2 * size), replace=False)
        base[outlier_idx] = np.random.normal(0, 10, size=len(outlier_idx))
        return base

    else:
        raise ValueError("Unsupported noise type")

def simulate_dataset_noise(noise_type,n_per_group):
    rows = []
    noise_vector = generate_noise(noise_type, size=n_groups * n_per_group)
    noise_counter = 0

    for j in range(n_groups):
        group_name = group_labels[j]
        group_boost = group_effects[j]

        for _ in range(n_per_group):
            logins_last_week = np.random.poisson(lam=5)
            previous_purchases = np.random.poisson(lam=2)
            viewed_target_category = np.random.binomial(1, p=0.5)
            discount_received = np.random.binomial(1, p=0.5)

            U = (
                -4.5
                + 0.1 * previous_purchases
                + 0.6 * viewed_target_category
                + 0.9 * discount_received
                + 3 * viewed_target_category * discount_received
                + 1.1 * group_boost * discount_received
                + 0.13 * previous_purchases**2
                + noise_vector[noise_counter]
            )
            noise_counter += 1

            y = 1 if U >0 else 0

            rows.append({
                "group_id": j,
                "group_label": group_name,
                "logins_last_week": logins_last_week,
                "previous_purchases": previous_purchases,
                "viewed_target_category": viewed_target_category,
                "discount_received": discount_received,
                "y": y
            })
    df_simulated = pd.DataFrame(rows)
    scaler = StandardScaler()
    df_simulated[["logins_last_week", "previous_purchases"]] = scaler.fit_transform(
    df_simulated[["logins_last_week", "previous_purchases"]])
    return df_simulated


df_simulated_cauchy_test = simulate_dataset_noise("cauchy",10000)
df_simulated_t_test = simulate_dataset_noise("t",10000)
df_simulated_contaminated_test = simulate_dataset_noise("contaminated",10000)



In [ ]:
df_simulated_contaminated_test["y"].value_counts()



In [ ]:
def run_experiment_loop2(
    seeds,
    feature_cols,
    noise_type_for_train="normal",
    npergroup=n_per_group,
    group=False, interaction=False, nonlinear=False,
    use_quasi=False, loss_kind="bce",
    draws=1000, tune=1000, target_accept=0.9
):
    results = []


    for seed in seeds:
        np.random.seed(seed)

        df_train = simulate_dataset_noise(noise_type_for_train,npergroup)
        group_idx_train = df_train["group_id"].values if group else None
        df_test = simulate_dataset_noise(noise_type_for_train, 10000)
        X_test = df_test[feature_cols].copy()
        y_test = df_test["y"].values
        group_idx_test = df_test["group_id"].values if group else None
        if interaction:
          X_test["interaction"] = X_test["viewed_target_category"] * X_test["discount_received"]
        if nonlinear:
          X_test["purchases_squared"] = X_test["previous_purchases"] ** 2
        X_test = X_test.values.astype("float64")

        model, eta, y = define_model(
            df_train, feature_cols,
            group_idx=group_idx_train,
            group=group, interaction=interaction, nonlinear=nonlinear
        )

        if use_quasi:
            trace = run_quasi_model(
                model, eta, y,
                loss_kind=loss_kind,
                draws=draws, tune=tune, target_accept=target_accept,
                return_inferencedata=True,     idata_kwargs={"log_likelihood": True}
            )
        else:
            trace = run_bayesian_model(
                model, eta, y,
                draws=draws, tune=tune, target_accept=target_accept,
                return_inferencedata=True,     idata_kwargs={"log_likelihood": True}
            )


        beta_da = trace.posterior["beta"].stack(sample=("chain", "draw"))

        if beta_da.ndim == 2:
          beta = beta_da.transpose("sample", ...).values

          eta = X_test @ beta.T

        else:
          beta = beta_da.transpose("sample", ...).values
          S, G, K = beta.shape

          beta_g = beta[:, group_idx_test, :]

          eta = np.einsum("snk,nk->ns", beta_g, X_test)

        p_samples = norm.cdf(eta)

        p_mean = p_samples.mean(axis=1)

        y_pred = (p_mean >= 0.5).astype(int)

        acc = accuracy_score(y_test, y_pred)
        logloss_val = log_loss(y_test, p_mean, labels=[0,1])
        brier = brier_score_loss(y_test, p_mean)

        results.append({
            "seed": seed,
            "acc": acc,
            "logloss": logloss_val,
            "brier": brier,
            "model_type": "quasi" if use_quasi else "bayes",
            "loss_kind": loss_kind if use_quasi else None
        })

    return pd.DataFrame(results)

## 1. Classical Bayesian model

In [ ]:
df_results_bayes_cauchy = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=False,
    noise_type_for_train="cauchy"
)

print(df_results_bayes_cauchy)

In [ ]:
df_results_bayes_t = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=False,
    noise_type_for_train="t"
)

print(df_results_bayes_t)

In [ ]:
df_results_bayes_conta = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=False,
    noise_type_for_train="contaminated"
)
print(df_results_bayes_conta)

## 2. Quasi Bayes model with bce loss function

In [ ]:
df_results_quasi_cauchy = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=True,
    noise_type_for_train="cauchy"
)

print(df_results_quasi_cauchy)


In [ ]:
df_results_quasi_t = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=True,
    noise_type_for_train="t"
)
print(df_results_quasi_t)


In [ ]:
df_results_quasi_conta = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=True,
    noise_type_for_train="contaminated"
)
print(df_results_quasi_conta)

## 3. SPH Bayes model

In [ ]:
df_results_sph_cauchy = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=True,
    noise_type_for_train="cauchy",loss_kind = "sph"
)

print(df_results_sph_cauchy)

In [ ]:
df_results_sph__t = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=True,
    noise_type_for_train="t",loss_kind = "sph"
)

print(df_results_sph__t)

In [ ]:
df_results_sph_conta = run_experiment_loop2(
    seeds=range(20),
    feature_cols=feature_cols,
    npergroup = 200,
        interaction=True,
        nonlinear=True,
        group=True,
    use_quasi=True,
    noise_type_for_train="contaminated",loss_kind = "sph"
)




print(df_results_sph_conta)

## 4. KSD Bayes model

In [ ]:
all_metrics = []
noise_type = "cauchy"
for seed in range(10):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type=noise_type,
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = noise_type,
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=True, nonlinear=True, group=True,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )
    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)

In [ ]:
all_metrics = []
noise_type = "t"
for seed in range(3):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type=noise_type,
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = noise_type,
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=True, nonlinear=True, group=True,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )
    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)

In [ ]:
all_metrics = []
noise_type = "contaminated"
for seed in range(10):

    np.random.seed(seed); torch.manual_seed(seed)
    df_train = simulate_dataset(
        noise_type=noise_type,
        n_per_group=200
    )
    df_test = simulate_dataset(
        noise_type = noise_type,
        n_per_group=10000
    )
    res = fit_ksd_bayes_nuts_ema_ensemble(
        df_train, df_test, feature_cols,
        interaction=True, nonlinear=True, group=True,
        n_outer=40, nuts_warmup=300, nuts_samples=30,
        beta_lr=0.01, target_accept_prob=0.90,
        device="cuda", verbose=True
    )
    all_metrics.append(res["metrics_test"])
    print(all_metrics)

# 집계
df = pd.DataFrame(all_metrics)
summary = df.agg(['mean','std','median'])
print(summary)
print(df)